# Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from plot_helper import Kanagawa2017_gap_profile

import dustpy

year = dustpy.constants.year
au = dustpy.constants.au

In [ ]:
def refinegrid(ri, r0, num=3):
    """Function to refine the radial grid

    Parameters
    ----------
    ri : array
        Radial grid
    r0 : float
        Radial location around which grid should be refined
    num : int, option, default : 3
        Number of refinement iterations

    Returns
    -------
    ri : array
        New refined radial grid"""
    if num == 0:
        return ri
    ind = np.argmin(r0 > ri) - 1
    indl = ind-num
    indr = ind+num+1
    ril = ri[:indl]
    rir = ri[indr:]
    N = (2*num+1)*2
    rim = np.empty(N)
    for i in range(0, N, 2):
        j = ind-num+int(i/2)
        rim[i] = ri[j]
        rim[i+1] = 0.5*(ri[j]+ri[j+1])
    ri = np.concatenate((ril, rim, rir))
    return refinegrid(ri, r0, num=num-1)

## Run Dustpy
Takes of the order of an hour

In [ ]:
bump = True

In [ ]:
ri = np.geomspace(2, 500, 200) * au
ri_ref = refinegrid(ri, 28.713 * au, 5)

In [ ]:
s = dustpy.Simulation()
s.ini.gas.alpha = 1e-3
s.ini.dust.vfrag = 1000.
s.grid.ri = ri_ref
s.initialize()

s.dust.v.rel.azi.save = False
s.dust.v.rel.brown.save = False
s.dust.v.rel.rad .save = False
s.dust.v.rel.tot .save = False
s.dust.v.rel.turb.save = False
s.dust.v.rel.vert.save = False
s.dust.kernel.save = False
s.dust.p.frag = False
s.dust.p.stick = False
s.t.snapshots = np.geomspace(1e2, 1e6, 50) * year

In [ ]:
s.writer.overwrite = True

In [ ]:
s.writer.datadir = 'data' + bump * '_bump'

In [ ]:
s.verbosity = 10

Bump parameters

In [ ]:
rp = 20. * au
h = np.interp(rp, s.grid.r, s.gas.Hp / s.grid.r)
q = 1e-3
hp = np.interp(rp, s.grid.r, s.gas.Hp) / rp
alpha0 = s.ini.gas.alpha

In [ ]:
if bump:
    finv = Kanagawa2017_gap_profile(s.grid.r, rp, q, hp, alpha0)
    p = 10
    finv_s = 2**(1 / p) / (np.ones_like(finv) + finv**(-p))**(1 / p)
    s.gas.alpha = np.ones_like(s.gas.alpha) * alpha0 / finv_s

    fig, ax = plt.subplots()
    ax.semilogy(s.grid.r / au, s.gas.Sigma * finv)
    ax.semilogy(s.grid.r / au, s.gas.Sigma * finv_s)
    ax.set_xlim(10, 35)
    ax.set_ylim(3e-1, 3e1)

In [ ]:
f, ax = plt.subplots()
ax.set_xlim(10, 40)
ax.set_ylim(0, 3)
ax.plot(ri / au, 1.0 * np.ones(len(ri)), '+')
ax.plot(ri_ref / au, 2.0 * np.ones(len(ri_ref)), '+')

In [ ]:
s.update()
s.run()